In [8]:
import pandas as pd

In [ ]:
import jwt
from jwt.utils import get_int_from_datetime
import uuid
import requests
from datetime import datetime, timedelta, timezone

# Constants
TOKEN_ROUTE = "https://kp-viewpoint-api.signalpath.com/v1/oauth/token"
PRIVATE_KEY_PATH = "test_key.pem"
AUDIENCE = "https://kp-signalpath.us.auth0.com/"
CLIENT_ID = "XwuLT9tzXzJmGxEqid8yAozgvRV7z19K"
KEY_ID = keyID  # your keyID

# Load private key
with open(PRIVATE_KEY_PATH, "rb") as fh:
    private_key = jwt.jwk_from_pem(fh.read())


def create_jwt(private_key, client_id, audience, key_id):
    instance = jwt.JWT()


# Current time
issued_at = get_int_from_datetime(datetime.now(timezone.utc))

# Expiration time (1 minute)
expiration_time = get_int_from_datetime(
    datetime.now(timezone.utc) + timedelta(minutes=1)
)
# JWT payload
payload = {
    "iss": client_id,
    "sub": client_id,
    "aud": audience,
    "iat": issued_at,
    "exp": expiration_time,
    "jti": str(uuid.uuid4()),
}

# JWT headers
headers = {"alg": "RS256", "kid": key_id}
token = instance.encode(
    payload=payload, key=private_key, alg="RS256", optional_headers=headers
)
return token


def main():
    # Create JWT
    jwt_token = create_jwt(private_key, CLIENT_ID, AUDIENCE, KEY_ID)
    # Setup JWT request data
    request_data = {
        "client_assertion_type": "urn:ietf:params:oauth:client-assertion-type:jwt-bearer",
        "client_assertion": jwt_token,
    }

    try:
        # Post JWT to token route
        response = requests.post(
            TOKEN_ROUTE, json=request_data, headers={"content-type": "application/json"}
        )
        response_data = response.json()

        # Get API response using access token
        api_response = requests.get(
            "https://kp-viewpoint-api.signalpath.com/v1/edp/site-study-detail",  # Provide the URL for the API request here
            headers={"authorization": f"Bearer {response_data['access_token']}"},
        )
        print(api_response.json())
    except requests.RequestException as error:
        print(f"An error occurred: {error}")


if __name__ == "__main__":
    main()

NameError: name 'keyID' is not defined